## Двухслойная нейронная сеть на numpy

In [ ]:
'''
Построение двухслойной нейронный сети для классификации цветков ириса
'''

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# sklearn здесь только, чтобы разделить выборку на тренировочную и тестовую
from sklearn.model_selection import train_test_split


### Шаг 1. Определение функций, которые понадобяться для обучения
# преобразование массива в бинарный вид результатов
def to_one_hot(Y):
    n_col = np.amax(Y) + 1
    binarized = np.zeros((len(Y), n_col))
    for i in range(len(Y)):
        binarized[i, Y[i]] = 1.
    return binarized

# преобразование массива в необходимый вид
def from_one_hot(Y):
    arr = np.zeros((len(Y), 1))

    for i in range(len(Y)):
        l = layer2[i]
        for j in range(len(l)):
            if(l[j] == 1):
                arr[i] = j+1
    return arr

# сигмоида и ее производная
def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_deriv(x):
    return sigmoid(x)*(1 - sigmoid(x))

# нормализация массива
def normalize(X, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(X, order, axis))
    l2[l2 == 0] = 1
    return X / np.expand_dims(l2, axis)


### Шаг 2. Подготовка тренировочных данных
# получения данных из csv файла. укажите здесь путь к файлу Iris.csv
iris_data = pd.read_csv("Iris.csv")
# print(iris_data.head()) # расскоментируйте, чтобы посмотреть структуру данных

# репрезентация данных в виде графиков
# g = sns.pairplot(iris_data.drop("Id", axis=1), hue="Species")
# plt.show() # расскоментируйте, чтобы посмотреть

# замена текстовых значений на цифровые
iris_data['Species'].replace(['Iris-setosa', 'Iris-virginica', 'Iris-versicolor'], [0, 1, 2], inplace=True)

# формирование входных данных
columns = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
x = pd.DataFrame(iris_data, columns=columns)
x = normalize(x.values)

# формирование выходных данных(результатов)
columns = ['Species']
y = pd.DataFrame(iris_data, columns=columns)
y = y.values
y = y.flatten()
y = to_one_hot(y)

# Разделение данных на тренировочные и тестовые
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)
   
best_accuracy = [0, '']
### Шаг 3. Обученние нейронной сети
# присваевание случайных весов
for neurons in [5, 6, 7]:
  w0 = 2*np.random.random((4, neurons)) - 1 # для входного слоя   - 4 входа, 3 выхода
  w1 = 2*np.random.random((neurons, 3)) - 1 # для внутреннего слоя - 5 входов, 3 выхода


  # скорость обучения (learning rate)
  for lr in [0.001, 0.01, 0.1, 1.0]:

    # процесс обучения
    for iter in [10000, 30000, 50000, 100000]:
      # массив для ошибок, чтобы потом построить график
      errors = []
      for i in range(iter): #100,000

          # прямое распространение(feed forward)
          layer0 = X_train
          layer1 = sigmoid(np.dot(layer0, w0))
          layer2 = sigmoid(np.dot(layer1, w1))

          # обратное распространение(back propagation) с использованием градиентного спуска
          layer2_error = y_train - layer2
          layer2_delta = layer2_error * sigmoid_deriv(layer2)
          
          layer1_error = layer2_delta.dot(w1.T)
          layer1_delta = layer1_error * sigmoid_deriv(layer1)
          
          w1 += layer1.T.dot(layer2_delta) * lr
          w0 += layer0.T.dot(layer1_delta) * lr
          
          error = np.mean(np.abs(layer2_error))
          errors.append(error)
          accuracy = (1 - error) * 100
          if accuracy > best_accuracy[0]:
            best_accuracy[0] = accuracy
            best_accuracy[1] = f'({neurons=} {lr=} {iter=} {accuracy=} {error=})'
      print(f'{neurons=} {lr=} {iter=} {accuracy=} {error=}')


### Шаг 4. Демонстрация полученных результатов
# черчение диаграммы точности в зависимости от обучения
# plt.plot(errors)
# plt.xlabel('Обучение')
# plt.ylabel('Ошибка')
# plt.show() # расскоментируйте, чтобы посмотреть 
        
print("Точность нейронной сети " + str(round(best_accuracy[0],2)) + "%", best_accuracy[1])

neurons=5 lr=0.001 iter=10000 accuracy=78.77006645542022 error=0.21229933544579788
neurons=5 lr=0.001 iter=30000 accuracy=86.377305568243 error=0.13622694431757
neurons=5 lr=0.001 iter=50000 accuracy=91.04353538165027 error=0.08956464618349727
neurons=5 lr=0.001 iter=100000 accuracy=97.14145840706742 error=0.028585415929325838
neurons=5 lr=0.01 iter=10000 accuracy=97.87107796614555 error=0.021289220338544544
neurons=5 lr=0.01 iter=30000 accuracy=98.01922799294925 error=0.01980772007050751
neurons=5 lr=0.01 iter=50000 accuracy=98.3074226862193 error=0.01692577313780697
neurons=5 lr=0.01 iter=100000 accuracy=99.15064940460903 error=0.008493505953909757
neurons=5 lr=0.1 iter=10000 accuracy=99.31624144516303 error=0.006837585548369788
neurons=5 lr=0.1 iter=30000 accuracy=99.43966837928592 error=0.005603316207140884


<ipython-input-3-7e0383fead47>:39: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


neurons=5 lr=0.1 iter=50000 accuracy=99.35661785689697 error=0.006433821431030231
neurons=5 lr=0.1 iter=100000 accuracy=98.64288264570676 error=0.013571173542932365
neurons=5 lr=1.0 iter=10000 accuracy=99.24257517092268 error=0.007574248290773259
neurons=5 lr=1.0 iter=30000 accuracy=99.94373832837383 error=0.0005626167162618421
neurons=5 lr=1.0 iter=50000 accuracy=99.97709105932945 error=0.00022908940670540644
neurons=5 lr=1.0 iter=100000 accuracy=99.9757223269662 error=0.0002427767303379324
neurons=6 lr=0.001 iter=10000 accuracy=79.89352442643059 error=0.20106475573569413
neurons=6 lr=0.001 iter=30000 accuracy=90.49166425044307 error=0.09508335749556934
neurons=6 lr=0.001 iter=50000 accuracy=95.39146947365244 error=0.046085305263475665
neurons=6 lr=0.001 iter=100000 accuracy=97.07443733835275 error=0.029255626616472477
neurons=6 lr=0.01 iter=10000 accuracy=97.5209115914009 error=0.024790884085990988
neurons=6 lr=0.01 iter=30000 accuracy=97.96950568382195 error=0.0203049431617806
neuro

Домашнее задание:

Задание 1. Попробуйте видоизменить параметры двухслойной нейронной сети таким образом, чтобы улучшить ее точность (число нейронов, эпох, слоев).

Задание 2. Проведите анализ - что приводит к ухудшению нейронной сети? Что приводит к увеличению ее точности?

1. Для улучшения точности было изменено число нейронов, эпох и скорость обучения.


2. У ухудшению нейронной сети приводит уменьшение числа нейронов, слоев, эпох. К увеличению точности приводит увеличение числа нейронов, слоев, эпох, но при обучении нужно отслеживать переобучение и слишком много слоев даст затухание градиента на начальных слоях.